In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import time
import re
import os
from tqdm import tqdm
import datetime as dt

In [2]:
def webcapture(url):
    headers = {
   'User-Agent': 'Mozilla/5.0',
   }
    session = requests.Session() 
    data = session.get(url, headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser') 
    return soup 

In [3]:
sdate = dt.date(2020, 1, 20)   
edate = dt.date(2020, 4, 11)   

delta = edate - sdate   
date_range = []
for i in range(delta.days + 1):
    day = sdate + dt.timedelta(days=i)
    date_range.append(str(day))

In [5]:
url_base = 'https://trendogate.com/placebydate/2352824/'
usa_trendings = pd.DataFrame()

for idx,date in tqdm(enumerate(date_range)):
    url = url_base+date
    soup = webcapture(url)
    trending_list = [li.get_text() for li in soup.find_all('li','list-group-item')]
    trending = []
    for trend in trending_list:
        trending.append(trend+'='+date)
    usa_trendings = pd.concat([usa_trendings,pd.DataFrame(trending)])
    time.sleep(2)
    
usa_trendings[['trending','date']] = usa_trendings.iloc[:,0].str.split('=',1,expand=True).iloc[:,0:2]

83it [07:32,  4.04s/it]


In [6]:
usa_trendings = usa_trendings.iloc[:,1:3]

In [8]:
os.chdir(r'C:\Users\Ananya\Documents\twitterData\covid19')
usa_trendings.to_csv('usa_trending.csv',index=False)

In [9]:
url = 'https://trendogate.com/placebydate/2352824/2020-04-05'

In [10]:
soup = webcapture(url)

In [11]:
woeid = [option.get('value') for option in soup.find_all('option')]
location = [option.get_text() for option in soup.find_all('option')]

In [12]:
usa_id = [re.findall(r'United States',str(loc)) for loc in location]
usa_idx = []
for idx,value in enumerate(usa_id):
    if value != []:
        usa_idx.append(idx)
    else:
        continue

In [13]:
usa_woeid = []
usa_location = []
for idx in usa_idx:
    usa_woeid.append(woeid[idx])
    usa_location.append(location[idx])

In [14]:
usa_df = pd.DataFrame(list(zip(usa_woeid,usa_location)),columns=['woeid','location'])
os.chdir(r'C:\Users\Ananya\Documents\twitterData\covid19')
usa_df.to_csv('usa_woeid_list.csv',index=False)